In [41]:
from sage.all import *
from fpylll import *
from g6k import *
from time import time
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


def is_smooth(x: int, P_list: list[int]):
    """Проверка на гладкость"""
    for p in P_list:
        while x % p == 0:
            x //= p
    return abs(x) == 1


def primes_factor(x: int, P_list: list[int]):
    """Разложение на степени простых множителей"""
    result = []
    for p in P_list:
        c = 0
        while x % p == 0:
            x //= p
            c += 1
        result.append(c)
    return result


def first_primes(n: int):
    """Генерация первых n простых чисел"""
    pr = 1
    P = []
    while len(P) < n:
        pr = next_prime(pr)
        P += [pr]
    return P


def short_vectors(A,d):
    """Нахождение коротких векторов"""
    A = IntegerMatrix.from_matrix(A)
    g6k = Siever(A)
    g6k.lll(0, d + 1)
    g6k.initialize_local(0, (d + 1) / 2, d + 1)
    while g6k.l > 0:
        g6k.extend_left(1)
        g6k()

    with g6k.temp_params(
            saturation_ratio=.95, 
            saturation_radius=1.75, 
            db_size_base=sqrt(1.7), 
            db_size_factor=3
        ):
        g6k()

    db = list(g6k.itervalues())
    vectors = []
    for x in db:
        v = A.multiply_left(x)
        vectors.append(v)
    return vectors

def shnorr(N: int, d: int, C: int, P_list: list[int]):
    while True:
        A, B, checks = [], [] , []
        while len(A) < d + 1:
            f = [*range(1, d + 1)]
            shuffle(f)
            if f not in checks: 
                checks.append(f)
            else: 
                continue
                
            A_ = diagonal_matrix([C * N * f[i] for i in range(d)] + [C * N * round(ln(N))], sparse=False)
            for i in range(d):
                A_[d, i] = C * N * round(ln(P_list[i]))

            z_vectors = short_vectors(A_, d)

            for i in z_vectors:
                z = [round(i[j] / (C * N * f[j])) for j in range(d)]

                u = 1
                k = 1
                for i in range(d):
                    assert z[i] in ZZ
                    if z[i] > 0:
                        u *= P_list[i]^z[i]
                    if z[i] < 0:
                        k *= P_list[i]^(-z[i])
                
                v = int(u - k * N)
                resA = [0] * d
                    
                if is_smooth(v, P_list) and len(A) < d + 1:
                    for i in range(d):
                        if z[i] > 0:
                            resA[i] = z[i]
                    if resA not in A:
                        A.append(resA)
                        B.append(primes_factor(v, P_list))

        M = []
        for i in range(d+1):
            m = []
            for j in range(d):
                m.append(int(A[i][j] + B[i][j]))
            M.append(m)
        M = IntegerMatrix.from_matrix(M)

        M = Matrix(ZZ, M)
        c_lst = kernel(Matrix(GF(2),M))

        for _ in c_lst:
            c = [int(i) for i in _]
            x, y = 1, 1
            for j in range(d):
                sx, sy = 0, 0
                for i in range(d + 1):
                    sx += c[i] * (A[i][j] + B[i][j]) / 2
                    sy += c[i] * A[i][j]
                x *= ((pow(P_list[j], sx,N))) % N
                y *= (pow(P_list[j], sy,N)) % N

            if x % N != y % N and x % N != -y % N:
                p = gcd(int(N), int(x + y))
                if p != 1:
                    q = N // p
                    return p, q

bits = 30
p = random_prime(pow(2, bits // 2), lbound=pow(2, bits // 2 - 1))
q = random_prime(pow(2, bits // 2), lbound=pow(2, bits // 2 - 1))
# p, q = 5381, 11821

N = p * q 
d = 30
print(f"N = {N}, битовая длина N = {N.bit_length()}")

P_list = first_primes(d)
print(f"p = {p}, q = {q}, d = {d}\nПервые {d} простых чисел = {P_list}")

С = randint(2**100, 2**101)
start_time = time()
p_find, q_find = shnorr(N, d, С, P_list)
end_time = time()

print(f"Время факторизации: {timedelta(seconds=int(end_time - start_time))}")
print(f"Результат факторизации: {p_find = }, {q_find = }")

N = 777624833, битовая длина N = 30
p = 26927, q = 28879, d = 30
Первые 30 простых чисел = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113]
